In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv as csv
import random



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

random.seed(42)

**Setup with algorithms**

In [ ]:
def newTable(data):
    max = 0
    count = 0
    
    with open(data, 'r', newline ='') as f:
        reader = csv.reader(f, delimiter=' ')
        for line in reader:
            count +=1
            
            for i in line:
                index = int(i.split(":")[0])
                
                if index > max:
                    max = index
    return np.zeros((count, max+1))

In [ ]:
def fillTable(table,df):
    with open(df, 'r') as f:
        reader = csv.reader(f, delimiter=" ")
        row = -1
        for i in reader:
            row +=1
            
            for o in i:
                data = o.split(":")
                
                if(int(data[0]) == 0 or int(data[0])==1):
                    table[row, 0] = int(data[0])
                else:
                    table[row, int(data[0])] = int(data[1])

    return table

In [ ]:
def createNPY(input,output):
    print(f"{input}")
    print(f"{output}")
    table = newTable(input)
    fillTable(table,input)
    print("Done")
    return table

In [ ]:
def predict(x,w,b):
  return x.dot(w) + b

def create_weights(length):
  w = np.zeros((length,1))
  for i in range(length):
    w[i] = random.uniform(-0.01,0.01)
  b = random.uniform(-0.01,0.01)
  return w,b

def update_weights(w, b, x, y, n):
  w = w.T
  
  indent = 1
  if y == 0:
    indent = -1
  w = w + (n * indent * x)
  b = b + (n * indent)
  return w.T, b

def splitdata(data):
  
  #data_x = data.iloc[:,1:] #columns
  #ata_y = data.iloc[:,0] #labels
  #for numpy
  data_x = data[:,1:] #columns
  data_y = data[:,0] #labels

  return data_x, data_y

def accuracy(labels, predictions):
  correct = 0

  for i in range(len(labels)):
    if labels[i] == predictions[i]:
      correct += 1

  return correct / float(len(labels)) * 100.0

In [ ]:
#x = features y= labels, n = learning rate, e = epochs
def simple_batch_perceptron_decay(x, y, n, e):

  #features = x.to_numpy(copy=True)
  #labels = y.to_numpy(copy=True)
  features = x
  labels = y
  

  length = features.shape[1]
  #print(length)
  w, b = create_weights(length) # TODO create_weights: this will create some value array of each w
  
  for epochs in range(e):

    n = n /(1+epochs)
    for i in range(features.shape[0]-1):
      y_prime = predict(features[i,0:length],w,b) #TODO create predict
    

      #y_i = 1, w^tx <=0 sign = -1
      #mistake on positive
      if y_prime >= 0 and labels[i] == 0:
        w, b = update_weights(w, b, features[i, 0:length],labels[i],n)
    
      #y_i = -1, w^tx >=0 sign = 1
      #mistake on negative
      elif y_prime <= 0 and labels[i] > 0:
        w, b = update_weights(w, b, features[i,0:length],labels[i],n)
  return w,b

In [ ]:
def simple_batch_perceptron(x, y, n, e):

  features = x
  labels = y

  length = features.shape[1]
  #print(length)
  w, b = create_weights(length) # TODO create_weights: this will create some value array of each w
  
  for epochs in range(e):
    for i in range(features.shape[0]-1):
      y_prime = predict(features[i,0:length],w,b) #TODO create predict
    

      #y_i = 1, w^tx <=0 sign = -1
      #mistake on positive
      if y_prime >= 0 and labels[i] < 0:
        w, b = update_weights(w, b, features[i, 0:length],labels[i],n)
    
      #y_i = -1, w^tx >=0 sign = 1
      #mistake on negative
      elif y_prime <= 0 and labels[i] > 0:
        w, b = update_weights(w, b, features[i,0:length],labels[i],n)

  return w,b

In [ ]:
def perceptron(x, w, b):
  length = x.shape[1]
  predictions = np.zeros((x.shape[0], 1))

  for i in range(len(x)):
    prediction = predict(x[i,0:length], w, b)
    #prediction = predict(x.iloc[i,0:length], w, b)
    #print(prediction)
    if prediction > 0:
      predictions[i] = 1
    else:
      predictions[i] = 0
  
  return predictions

In [ ]:
def test_perceptron(test_folds,train_folds,n,epochs):
  
  #split train folds
  train_folds_x,train_folds_y = splitdata(train_folds)
  
  #start training
  w,b = simple_batch_perceptron(train_folds_x,train_folds_y,n,epochs)
  
  #test
  test_folds_x,test_folds_y = splitdata(test_folds)
  s = perceptron(test_folds_x,w,b)
  #percent = accuracy(test_folds_y,s)


  #print(f"n: {n}, epochs: {epochs}, accuracy: {percent}")
    
  return s

In [ ]:
def test_perceptron_simple(test_folds,train_folds,n,epochs):
  
  #split train folds
  train_folds_x,train_folds_y = splitdata(train_folds)
  
  #start training
  w,b = simple_batch_perceptron_decay(train_folds_x,train_folds_y,n,epochs)
  
  #test
  test_folds_x,test_folds_y = splitdata(test_folds)
  s = perceptron(test_folds_x,w,b)
  #percent = accuracy(test_folds_y,s)


  #print(f"n: {n}, epochs: {epochs}, accuracy: {percent}")
    
  return s

In [ ]:
def fillTableWOint(table,df):
    with open(df, 'r') as f:
        reader = csv.reader(f, delimiter=" ")
        row = -1
        for i in reader:
            row +=1
            
            for o in i:
                data = o.split(":")
                
                if(int(data[0]) == 0 or int(data[0])==1):
                    table[row, 0] = int(data[0])
                else:
                    table[row, int(data[0])] = data[1]

    return table

def createNPYWOint(input,output):
    print(f"{input}")
    print(f"{output}")
    table = newTable(input)
    fillTableWOint(table,input)
    print("Done")
    return table

**Testing bag of words dataset with simple batch perceptron**

In [ ]:
input_file = '/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.train.libsvm'
testDataCSV = '/kaggle/working/bow.train.csv'

train = createNPY(input_file,testDataCSV)

input_file = '/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.test.libsvm'
testDataCSV = '/kaggle/working/bow.test.csv'

test = createNPY(input_file,testDataCSV)

#-----TO DO-----
# check which perceptron algo/learning rate/epochs is best for each one against training and testing data, then train and test against eval

#test_perceptron(test,train,1,20)
#test_perceptron(test,train,.1,20)
#test_perceptron(test,train,.01,20)




input_file = '/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.eval.anon.libsvm'
testDataCSV = '/kaggle/working/bow.eval.anon.csv'

evalTable = createNPY(input_file,testDataCSV)



s = test_perceptron(evalTable,train,1,10)
print(s.shape)


In [ ]:
id = '/kaggle/input/uofu-ml-fall-2020/project_data/data/eval.ids'
example = np.genfromtxt(id, delimiter = '\n')
example = example.reshape(s.shape[0], 1)

output = np.concatenate((example, s), axis = 1).astype(int)
  
pd.DataFrame(output).to_csv("/kaggle/working/bow.eval.prediction.csv", header=["example_id", "label"], index=None)




**Testing tfidf dataset with simple batch perceptron**

In [ ]:
# This is a dumby test, going to use glove instead
input_file = '/kaggle/input/uofu-ml-fall-2020/project_data/data/tfidf/tfidf.train.libsvm'
testDataCSV = '/kaggle/working/tfidf.train.csv'

train = createNPYWOint(input_file,testDataCSV)

input_file = '/kaggle/input/uofu-ml-fall-2020/project_data/data/tfidf/tfidf.eval.anon.libsvm'
testDataCSV = '/kaggle/working/tfidf.test.csv'

evalTable = createNPYWOint(input_file,testDataCSV)



#-----TO DO-----
# check which perceptron algo/learning rate/epochs is best for each one against training and testing data, then train and test against eval

#s = test_perceptron(test,train,1,20)
#test_perceptron(test,train,.1,20)
#test_perceptron(test,train,.01,20)
#print(s.shape)

s = test_perceptron(evalTable,train,1,10)



In [ ]:
id = '/kaggle/input/uofu-ml-fall-2020/project_data/data/eval.ids'
example = np.genfromtxt(id, delimiter = '\n')
example = example.reshape(s.shape[0], 1)

print(example.shape[1])
print(s)
output = np.concatenate((example, s), axis = 1).astype(int)
  
pd.DataFrame(output).to_csv("/kaggle/working/tfidf.eval.prediction.csv", header=["example_id", "label"], index=None)

****Testing tfidf with simple batch perceptron decay****

In [ ]:

input_file = '/kaggle/input/uofu-ml-fall-2020/project_data/data/tfidf/tfidf.train.libsvm'
testDataCSV = '/kaggle/working/tfidf.train.csv'

train = createNPYWOint(input_file,testDataCSV)

input_file = '/kaggle/input/uofu-ml-fall-2020/project_data/data/tfidf/tfidf.eval.anon.libsvm'
testDataCSV = '/kaggle/working/tfidf.test.csv'

evalTable = createNPYWOint(input_file,testDataCSV)


s = test_perceptron_simple(evalTable,train,.01,10)

id = '/kaggle/input/uofu-ml-fall-2020/project_data/data/eval.ids'
example = np.genfromtxt(id, delimiter = '\n')
example = example.reshape(s.shape[0], 1)

output = np.concatenate((example, s), axis = 1).astype(int)
  
pd.DataFrame(output).to_csv("/kaggle/working/tfidf.eval.prediction_simple.csv", header=["example_id", "label"], index=None)
